## Imports

In [1]:
from pymongo import MongoClient
from pprint import pprint

## Connect to mongo database

In [2]:
# Connect to Mongo server
host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Connect to mongo db
db = client.denormalised

In [5]:
db.list_collection_names()

['Application_Countries',
 'Warehouse_Colors',
 'Sales_OrderLines',
 'Warehouse_StockItemTransactions',
 'Application_StateProvinces',
 'Sales_Invoices',
 'Sales_Orders',
 'Purchasing_PurchaseOrders',
 'customers_payments',
 'Application_DeliveryMethods',
 'Application_Cities',
 'Application_TransactionTypes',
 'Warehouse_StockItems',
 'Sales_InvoiceLines',
 'Warehouse_StockItemStockGroups',
 'Purchasing_PurchaseOrderLines',
 'Purchasing_SupplierCategories',
 'Application_PaymentMethods',
 'Sales_Customers',
 'Sales_CustomerCategories',
 'Purchasing_SupplierTransactions',
 'Application_People',
 'Warehouse_PackageTypes',
 'Purchasing_Suppliers',
 'reviews',
 'Warehouse_StockGroups',
 'Sales_CustomerTransactions']

The web team needs:  

- From which state province are our suppliers from?

In [38]:
query_1 = {
    '$project':{
                'supplier':'$SupplierName',
                'state_province':'$DeliveryCity.StateProvince.StateProvinceName'
    }
}



pipeline = [query_1]

r = db.Purchasing_Suppliers.aggregate(pipeline)

result = list(r)

result

[{'_id': 1, 'supplier': 'A Datum Corporation', 'state_province': 'Indiana'},
 {'_id': 2, 'supplier': 'Contoso, Ltd.', 'state_province': 'Washington'},
 {'_id': 3,
  'supplier': 'Consolidated Messenger',
  'state_province': 'California'},
 {'_id': 4, 'supplier': 'Fabrikam, Inc.', 'state_province': 'Kentucky'},
 {'_id': 5,
  'supplier': 'Graphic Design Institute',
  'state_province': 'Missouri'},
 {'_id': 6, 'supplier': 'Humongous Insurance', 'state_province': 'Tennessee'},
 {'_id': 7, 'supplier': 'Litware, Inc.', 'state_province': 'California'},
 {'_id': 8, 'supplier': 'Lucerne Publishing', 'state_province': 'Tennessee'},
 {'_id': 9, 'supplier': 'Nod Publishers', 'state_province': 'North Carolina'},
 {'_id': 10,
  'supplier': 'Northwind Electric Cars',
  'state_province': 'New Jersey'},
 {'_id': 11, 'supplier': 'Trey Research', 'state_province': 'South Dakota'},
 {'_id': 12, 'supplier': 'The Phone Company', 'state_province': 'Minnesota'},
 {'_id': 13, 'supplier': 'Woodgrove Bank', 'stat

- From which state province are the customers who have a higher credit limit?

In [125]:
db.Sales_Customers.create_index(
    [('CreditLimit', 1)],
    name='CreditLimit', 
)

'CreditLimit'

In [126]:
#top 5 creditlimit customers's state provinces
query_1 = {
    '$match':{  'CreditLimit':  {"$exists": True}  }
}


query_2 = {'$project':
          {'credit_limit':'$CreditLimit',
          'state_province':'$DeliveryCity.StateProvince.StateProvinceName'}}

query_3 = {'$sort':
          {'credit_limit':-1}}

query_4 = {'$limit':5}


pipeline = [query_1, query_2, query_3, query_4]

r = db.Sales_Customers.aggregate(pipeline)

result = list(r)

result

[{'_id': 492, 'credit_limit': 4630.5, 'state_province': 'Texas'},
 {'_id': 170, 'credit_limit': 4200.0, 'state_province': 'Tennessee'},
 {'_id': 605, 'credit_limit': 4095.0, 'state_province': 'North Carolina'},
 {'_id': 500, 'credit_limit': 4095.0, 'state_province': 'Wyoming'},
 {'_id': 613, 'credit_limit': 4000.0, 'state_province': 'Oregon'}]

In [127]:
#top 5 state provinces with higher average credit limit
query_1 = {
    '$match':{  'CreditLimit':  {"$exists": True}  }
}


query_2 = {'$project':
          {'credit_limit':'$CreditLimit',
          'state_province':'$DeliveryCity.StateProvince.StateProvinceName'}}

query_3 = {'$group':
          {'_id':'$state_province',
          'avg_creditLimit':{'$avg':'$credit_limit'}}}


query_4 = {'$sort':
          {'avg_creditLimit':-1}}

query_5 = {'$limit':5}


pipeline = [query_1, query_2, query_3, query_4, query_5]

r = db.Sales_Customers.aggregate(pipeline)

result = list(r)

result

[{'_id': 'Tennessee', 'avg_creditLimit': 3157.5},
 {'_id': 'Arkansas', 'avg_creditLimit': 2883.125},
 {'_id': 'Indiana', 'avg_creditLimit': 2832.9166666666665},
 {'_id': 'Oregon', 'avg_creditLimit': 2823.75},
 {'_id': 'Wyoming', 'avg_creditLimit': 2799.1666666666665}]

Partnerships: 

 - Would like to know what’s the most common payment type?

In [122]:
db.Purchasing_SupplierTransactions.create_index(
    [('PaymentMethod.PaymentMethodName', 1)],
    name='PaymentMethodName', 
)

'PaymentMethodName'

In [45]:
# We checked that all the payment methods in Purchasing_SupplierTransactions are also in Sales_CustomerTransactions so we can 
# match collections
query_1 = {
    '$group': {
        '_id' : '$PaymentMethod.PaymentMethodName', 
        'count' : {'$sum' : 1}
    }
}

query_2 = {
    "$lookup":{
        "from": "Purchasing_SupplierTransactions",
        "localField": "_id",
        "foreignField": "PaymentMethod.PaymentMethodName",
        "as": "supplier_payments"
     }
}

query_3 = {
    "$unwind": "$supplier_payments"
}

query_4 = {
    '$group': {
        '_id' : '$_id',
        'cust_count' : { '$first': '$count' },
        'supp_count' : {'$sum' : 1}
    }
}

query_5 = {
    '$project': {
        '_id' : 0,
        'payment_type' : '$_id',
        'total_count': { '$sum': [ "$cust_count", "$supp_count" ] }
    }
}

query_6 = {
    '$sort': {'year': 1}
}

limit = {'$count':1}
pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.Sales_CustomerTransactions.aggregate(pipeline)

result = list(r)

result

[{'payment_type': 'EFT', 'total_count': 54397}]

In [165]:
#most common name over the costumers

query_1 = {
    '$project':{
                '_id':False,
                'payment_type':'$PaymentMethod.PaymentMethodName'
    }
}

query_2 = {
    '$group': {
        '_id' : '$payment_type', 
        'count' : {'$sum' : 1}            
}
}

# query_3 = {'$out':'customers_payments'}
          
pipeline = [query_1, query_2]#, query_3]

r = db.Sales_CustomerTransactions.aggregate(pipeline)

result = list(r)

result

[{'_id': 'EFT', 'count': 51959}, {'_id': None, 'count': 41}]

In [162]:
#most common over suppliers
query_1 = {
    '$project':{
                '_id':False,
                'payment_type':'$PaymentMethod.PaymentMethodName'
    }
}

query_2 = {
    '$group': {
        '_id' : '$payment_type', 
        'count' : {'$sum' : 1}            
}
}


query_3 = {'$sort':
          {'count':-1}}




# query_4 = {
#         '$lookup': {
#            "from": 'customer_payments',
#            "localField": "_id",
#            "foreignField": "_id",
#            "as": "customer_purchases"
#         }}


pipeline = [query_1, query_2, query_3, query_4]

r = db.Purchasing_SupplierTransactions.aggregate(pipeline)

result = list(r)

result

[{'_id': 'EFT', 'count': 2438, 'customer_purchases': []}]

- Which supplier of Novelty Goods Supplier as the most transactions?

In [43]:
db.Purchasing_SupplierTransactions.create_index(
    [('SupplierID', 1)],
    name='SupplierID', 
)

'SupplierID'

In [44]:
db.Purchasing_Suppliers.create_index(
    [('SupplierID', 1)],
    name='SupplierID', 
)

'SupplierID'

In [45]:
query_1 = {
    '$match':{
                'SupplierID':{'$in' : list(db.Purchasing_Suppliers.distinct('SupplierID',
                                    {'SupplierCategory.SupplierCategoryName':'Novelty Goods Supplier'}))
    }
}}

query_2 = {
    '$project':{
                '_id':False,
                'supplier_id':'$SupplierID'
    }
}

query_3 = {
    '$group': {
        '_id': '$supplier_id', 
        'count' : {'$sum' : 1}            
}
}

query_4 = {
    '$sort': {
        'count' : -1}            
}

query_5 =     {
        '$lookup': {
           "from": "Purchasing_Suppliers",
           "localField": "_id",
           "foreignField": "SupplierID",
           "as": "supplier_info"
        }}

query_6 = {
    '$project' :{
        '_id' : '$supplier_info.SupplierName',
        'count' : '$count'
    }
}


pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.Purchasing_SupplierTransactions.aggregate(pipeline)

result = list(r)

result

[{'_id': ['Graphic Design Institute'], 'count': 16},
 {'_id': ['A Datum Corporation'], 'count': 7},
 {'_id': ['The Phone Company'], 'count': 7},
 {'_id': ['Contoso, Ltd.'], 'count': 2}]

- To know which items get ordered together the most?

In [46]:
db.Sales_OrderLines.create_index(
    [('OrderID', 1)],
    name='OrderID', 
)

'OrderID'

In [56]:
db.Sales_OrderLines.create_index(
    [('Description', 1)],
    name='Description', 
)

'Description'

In [128]:
query_1 =     {
        '$lookup': {
           "from": "Sales_OrderLines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "joint_orders"
        }}



query_2 = {
    '$unwind': '$joint_orders'
}

query_3 = {
    '$unwind': '$joint_orders.Description'
}
    




query_4 = {'$project':
          {'_id':False,
          'ol1':'$Description',
          'ol2':'$joint_orders.Description',
           "Eq": {"$eq":["$Description","$joint_orders.Description"]}
            }}

query_5 = {'$match':
            {"Eq": False}}


query_6 = {'$group':
          {'_id': 
           {'ol1':'$ol1',
            'ol2':'$ol2'   },
           'count': { '$sum': 1 }}}

query_7 = {'$sort':{'count':-1}}

query_8 = {'$limit': 30}


pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8]

r = db.Sales_OrderLines.aggregate(pipeline)

result = list(r)

result

[{'_id': {'ol1': 'DBA joke mug - I will get you in order (White)',
   'ol2': 'Animal with big feet slippers (Brown) M'},
  'count': 15},
 {'_id': {'ol1': 'Animal with big feet slippers (Brown) M',
   'ol2': 'DBA joke mug - I will get you in order (White)'},
  'count': 15},
 {'_id': {'ol1': 'Black and orange handle with care despatch tape  48mmx75m',
   'ol2': 'DBA joke mug - two types of DBAs (Black)'},
  'count': 14},
 {'_id': {'ol1': 'DBA joke mug - two types of DBAs (Black)',
   'ol2': 'Black and orange handle with care despatch tape  48mmx75m'},
  'count': 14},
 {'_id': {'ol1': 'DBA joke mug - you might be a DBA if (White)',
   'ol2': 'RC vintage American toy coupe with remote control (Black) 1/50 scale'},
  'count': 13},
 {'_id': {'ol1': '10 mm Anti static bubble wrap (Blue) 10m',
   'ol2': 'Dinosaur battery-powered slippers (Green) M'},
  'count': 13},
 {'_id': {'ol1': 'Superhero action jacket (Blue) 4XL',
   'ol2': 'IT joke mug - that behavior is by design (White)'},
  'count': 